In [1]:
!pip install rapidfuzz

In [2]:
import pandas as pd
import matplotlib.pyplot as plt #visualisasi data
import numpy as np #proses nilai numerik
import seaborn as sns #visualisasi data
import scipy.stats as stats
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Format connection string
username = "postgres"
password = "Fajrimughni9"
host = "localhost"
port = "5432"
database = "Finpro Master DS29"

# Buat engine SQLAlchemy
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{database}")

# Dataset

In [4]:
df_zomato = pd.read_csv("selected_zomato_indonesia.csv")
df_nutrition = pd.read_csv("selected_nutrition.csv")
df_recipe = pd.read_csv("selected_recipe.csv")
df_consumption = pd.read_csv("df_consumption_cleaned.csv")

# Pre-processing Data

In [5]:
df_consumption.shape

(710, 28)

In [6]:
df_recipe.shape

(1916, 4)

In [7]:
df_nutrition.shape

(1346, 6)

In [8]:
df_recipe.head(5)

,title,ingredients,steps,loves
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4


In [9]:
df_nutrition.head(5)

,id,name,calories,proteins,fat,carbohydrate
0,1,Abon,280.0,9.2,28.4,0.0
1,2,Abon haruwan,513.0,23.7,37.0,21.3
2,3,Agar-agar,0.0,0.0,0.2,0.0
3,4,Akar tonjong segar,45.0,1.1,0.4,10.8
4,5,Aletoge segar,37.0,4.4,0.5,3.8


## Mapping Title/Nama Resep

In [10]:
# Buat vectorizer
vectorizer = TfidfVectorizer().fit(df_recipe['title'].tolist() + df_nutrition['name'].tolist())

In [11]:
# Transform judul dan nama nutrisi
recipe_vecs = vectorizer.transform(df_recipe['title'])
nutrition_vecs = vectorizer.transform(df_nutrition['name'])

In [12]:
# Hitung similarity
similarity_matrix = cosine_similarity(recipe_vecs, nutrition_vecs)

In [13]:
similarity_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(1916, 1346))

In [14]:
# Ambil hasil paling mirip
best_matches = similarity_matrix.argmax(axis=1)
match_scores = similarity_matrix.max(axis=1)

In [15]:
# Masukkan ke DataFrame
df_recipe['matched_name'] = df_nutrition.loc[best_matches, 'name'].values
df_recipe['match_score'] = match_scores * 100

## Profil Konsumen

In [16]:
df_consumption.head()

,a1_city,a2_province,b3_gender,b4_age,b5_religion,b6_ethnic,b7_education_level,b8_lived_in_urban_area_years,b9_occupation,b10_social_clas,...,e19_religious_value_in_eating_places,e20_religious_value_in_consumption_pattern,e21_custom_value_in_type_of_food,e22_custom_value_in_eating_places,e23_custom_value_in_consumption_pattern,24_practice_of_food_consumption_based_on_religious_rules,25_practice_of_food_consumption_based_on_custom_rules,e26_buying_food_product_in_the_right_place_based_on_religious_rules,e27_type_of_dishes_variation_based_on_religious_rules,e28_practice_of_consuming_traditional_food
0,Jakarta,DKI Jakarta,Female,31,Islam,Java,Senior High School,31,Private Employees,Lower Class,...,Neither agree nor disagree,Agree,Strongly Agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Disagree,Neither agree nor disagree,Agree,Agree
1,Jakarta,DKI Jakarta,Female,36,Islam,Java,Bachelor degree,32,Private Employees,Lower Class,...,Agree,Agree,Strongly Agree,Agree,Agree,Disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Strongly Agree
2,Jakarta,DKI Jakarta,Male,24,Islam,Java,Senior High School,24,Private Employees,Middle Class,...,Agree,Neither agree nor disagree,Neither agree nor disagree,Disagree,Agree,Disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree
3,Jakarta,DKI Jakarta,Female,63,Islam,Other,Senior High School,32,Private Employees,Lower Class,...,Agree,Neither agree nor disagree,Agree,Neither agree nor disagree,Agree,Agree,Neither agree nor disagree,Disagree,Neither agree nor disagree,Agree
4,Jakarta,DKI Jakarta,Male,43,Protestant,Batak,Primary School,43,Government Employees,Lower Class,...,Strongly Agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Strongly Agree,Strongly Agree,Agree,Disagree,Agree,Agree


In [17]:
df_consumption.columns

Index(['a1_city', 'a2_province', 'b3_gender', 'b4_age', 'b5_religion',
       'b6_ethnic', 'b7_education_level', 'b8_lived_in_urban_area_years',
       'b9_occupation', 'b10_social_clas', 'b11_household_member',
       'c12_household_income_monthly_in_million_idr',
       'c13_time_of_income_receipt',
       'c14_household_expenditure_monthly_in_million_idr',
       'c15_%_monthly_expenditure_for_food', 'd16_health_factor',
       'd17_price_factor', 'e18_religious_value_in_type_of_food',
       'e19_religious_value_in_eating_places',
       'e20_religious_value_in_consumption_pattern',
       'e21_custom_value_in_type_of_food', 'e22_custom_value_in_eating_places',
       'e23_custom_value_in_consumption_pattern',
       '24_practice_of_food_consumption_based_on_religious_rules',
       '25_practice_of_food_consumption_based_on_custom_rules',
       'e26_buying_food_product_in_the_right_place_based_on_religious_rules',
       'e27_type_of_dishes_variation_based_on_religious_rules',
   

In [18]:
# Hapus prefix angka dan huruf di depan underscore
df_consumption.columns = df_consumption.columns.str.replace(r'^[a-zA-Z0-9]+_', '', regex=True)

In [19]:
df_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 28 columns):
 #   Column                                                           Non-Null Count  Dtype 
---  ------                                                           --------------  ----- 
 0   city                                                             710 non-null    object
 1   province                                                         710 non-null    object
 2   gender                                                           710 non-null    object
 3   age                                                              710 non-null    int64 
 4   religion                                                         710 non-null    object
 5   ethnic                                                           710 non-null    object
 6   education_level                                                  710 non-null    object
 7   lived_in_urban_area_years                            

In [20]:
# Tambahkan age_group
def get_age_group(age):
    if 18 <= age <= 25:
        return '18–25'
    elif 26 <= age <= 35:
        return '26–35'
    elif 36 <= age <= 45:
        return '36–45'
    elif 46 <= age <= 55:
        return '46–55'
    else:
        return '56+'

In [21]:
df_consumption['age_group'] = df_consumption['age'].apply(get_age_group)

In [22]:
# Buat preferensi langsung per baris
df_consumption['prefer_traditional'] = df_consumption['practice_of_consuming_traditional_food'].apply(lambda x: 'Yes' if x == 'Agree' else 'No')
df_consumption['prefer_healthy'] = df_consumption['health_factor'].apply(lambda x: 'Yes' if x == 'Agree' else 'No')

In [23]:
# Pilih kolom akhir profil
consumer_profile = df_consumption[['gender', 'age_group', 'prefer_traditional', 'prefer_healthy']].copy()

# Lihat contoh
print(consumer_profile.head())

   gender age_group prefer_traditional prefer_healthy
0  Female     26–35                Yes             No
1  Female     36–45                 No             No
2    Male     18–25                 No             No
3  Female       56+                Yes             No
4    Male     36–45                Yes             No


In [24]:
consumer_profile.isnull().values.any()

np.False_

# Model

In [25]:
df_recipe.shape

(1916, 6)

In [26]:
df_recipe.sample(5)

,title,ingredients,steps,loves,matched_name,match_score
1251,Ayam Goreng Lengkuas,1/2 ekor ayam potong 4/6 sesuai selera--2-3 ru...,Cuci bersih ayam lalu lumuri dengan jeruk nipi...,9,Boros Laja (lengkuas),38.691339
522,Siomay ayam,1/2 kg ayam fillet--3 siung bawang putih--1 sd...,Giling daging fillet (sy pakai blender) bersam...,7,Siomay,96.534554
1485,Siomay ayam Simpel,1/4 daging ayam--1/2 sdt lada--Secukupnya peny...,"Haluskan daging ayam, saya pakai blender,. Jan...",2,Siomay,69.453058
723,Nasi Ayam Katsu Sayuran,2 buah Dada ayam fillet tanpa kulit dan tulang...,"Dada ayam, kanan dan kiri. Lumuri dengan tepun...",5,Nasi,48.553310
1530,Chicken steak ala akari papa,"200 gram daging ayam paha--secukupnya garam, l...",Paha ayam dicuci dan di lap dengan tissue dapu...,116,Chicken teriyaki masakan,12.713388


In [27]:
# Hapus baris yang matched_name-nya NaN
df_recipe_cleaned = df_recipe[df_recipe['matched_name'].notna()].reset_index(drop=True)

In [28]:
df_recipe_cleaned.shape

(1916, 6)

In [29]:
df_recipe_cleaned.sample(5)

,title,ingredients,steps,loves,matched_name,match_score
1461,Chicken Veggie Rolade (rolade ayam sayur) menu...,1/2 kg dada ayam utuh minta di fillet--250 gr ...,"Cuci bersih ayam, lumuri dgn jeruk nipis & bum...",6,Sate Kulit,16.047098
1231,Tum Ayam,500 gr Ayam Fillet--250 ml Santan Kental--3 si...,"Cincang kasar ayam fillet, sisihkan pada wadah...",4,Ayam,18.964026
1155,Ayam geprek Sambal Kosek,2-4 potong ayam goreng--Saya disini pakai ayam...,"Campur semua bahan sambal kecuali minyak, uleg...",14,Ayam,14.687833
141,Bakso ayam,500 gr daging ayam--250 gr tepung sagu--secuku...,Hancurkan(sy pke gilingan daging) daging ayam ...,5,Bakso,95.849882
432,Bubur oat ayam tahu,3 sdm oat instant merah--2 bawang putih--Sedik...,Larutkan 3 sdm oat dgn air panas. Sisihkan--Se...,4,Bubur,53.724902


In [30]:
df_nutrition.head(5)

,id,name,calories,proteins,fat,carbohydrate
0,1,Abon,280.0,9.2,28.4,0.0
1,2,Abon haruwan,513.0,23.7,37.0,21.3
2,3,Agar-agar,0.0,0.0,0.2,0.0
3,4,Akar tonjong segar,45.0,1.1,0.4,10.8
4,5,Aletoge segar,37.0,4.4,0.5,3.8


In [31]:
consumer_profile.head(5)

,gender,age_group,prefer_traditional,prefer_healthy
0,Female,26–35,Yes,No
1,Female,36–45,No,No
2,Male,18–25,No,No
3,Female,56+,Yes,No
4,Male,36–45,Yes,No


## Mapping Ulang

Mapping yang mempertimbangkan nilai kalori dan nama nutrisi serta judul resep,kemudian menggunakan pendekatan TF-IDF. Menambahkan informasi jenis-jenis nutrisi yang terkandung dalam resep (contoh: "Ayam Betutu Daun Kemangi") ke df_recipe

jika memisahkan dari awal lagi untuk nama-nama di df_recipe['name'] agar dapat ditambahkan nutrisinya kemudian dijumlahkan total kalorinya

# Memisahkan Nama Resep dengan Named Entity Matching dari Bahan Pangan di Nutrisi

In [32]:
# Ambil semua nama bahan dari df_nutrition
ingredient_list = df_nutrition['name'].str.lower().unique().tolist()

def split_by_ingredients(text, ingredient_list):
    text = text.lower()
    found = [ing for ing in ingredient_list if ing in text]
    return list(set(found)) if found else [text]

# Terapkan ke df_recipe
df_recipe['split_titles'] = df_recipe['title'].apply(lambda x: split_by_ingredients(x, ingredient_list))

In [33]:
df_recipe[['title', 'split_titles']].sample(5)

,title,split_titles
366,Tongseng Ayam,[ayam]
368,Ayam goreng tepung dengan saus gochujang,[ayam]
162,Crispy Chicken,[crispy chicken]
454,Ayam kecap rempah,"[ayam, kecap]"
98,Soto ayam special,[ayam]


Kayanya gacocok deh antara judul resep dengan nutrisi di makanan?


Jadi pakai apa yah?

In [34]:
df_recipe['main_ingredient'] = df_recipe['split_titles'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

In [35]:
df_recipe.sample(5)

,title,ingredients,steps,loves,matched_name,match_score,split_titles,main_ingredient
1247,Tahu ayam suwir,Tahu kuning (potong kotak-kotak) dan goreng--A...,"Tumis duo bawang hingga harum, dan bawang bomb...",1,Suwir-suwir,72.197533,"[ayam, uwi, tahu]",ayam
558,Garang asam ayam,"1/2 kg sayap ayam--1 batang wortel,potong dadu...","Rebus ayam dengan sedikit garam, buang lemakny...",5,Terong Asam,38.947382,[ayam],ayam
233,Pindang ceker ayam 🐣,500 gr ceker ayam--Secukup nya daun bawang--Se...,Cuci sampai bersih ceker ayam lalu rebus sampa...,2,Pindang Ikan Banjar,44.616522,[ayam],ayam
1438,Ayam Geprek Sambal Ijo,1/2 kg Ayam🐓 (saya pake dada ayam)--3/4 Bgks R...,Bersihkan Ayam dan lumuri jeruk nipis spaya ti...,6,Kacang Ijo,52.121961,[ayam],ayam
463,Ayam penyet sangar,1/4 dada ayam--1 bks masako sapi--bahan sambal...,Rebus ayam dengan masako sampai air meresap ti...,3,Ayam,15.253454,[ayam],ayam


In [36]:
df_recipe['main_ingredient'].value_counts()

main_ingredient
ayam                                        1688
keju                                          19
[notitle]                                     15
chicken katsu                                 11
chicken teriyaki                               5
                                            ... 
chicken crispy wings saus barbeque pedas       1
popcorn chicken                                1
chicken wings ala ph                           1
fire chicken richeese kw                       1
chicken stroganoff with mushroom               1
Name: count, Length: 166, dtype: int64

In [37]:
df_nutrition['name_clean'] = df_nutrition['name'].str.lower().str.strip()
df_recipe['main_ingredient_clean'] = df_recipe['main_ingredient'].str.lower().str.strip()

In [38]:
calorie_map = df_nutrition.set_index('name_clean')['calories'].to_dict()
df_recipe['main_ingredient_calories'] = df_recipe['main_ingredient_clean'].map(calorie_map)

In [39]:
df_recipe.columns

Index(['title', 'ingredients', 'steps', 'loves', 'matched_name', 'match_score',
       'split_titles', 'main_ingredient', 'main_ingredient_clean',
       'main_ingredient_calories'],
      dtype='object')

In [40]:
df_recipe.sample(5)

,title,ingredients,steps,loves,matched_name,match_score,split_titles,main_ingredient,main_ingredient_clean,main_ingredient_calories
1527,Tumis Jamur Tiram Brokoli Mix Pentol Ayam,2 Bungkus Jamur Tiram--1 Buah Brokoli Berukura...,Iris jamur dan brokoli sesuai selera--Kemudian...,5,Jamur tiram segar,48.125164,[ayam],ayam,ayam,302.0
1585,Ayam ungkep kuning,1 ekor ayam potong sesuai selera--3 lembar dau...,Cuci ayam hingga bersih beri jeruk nipis biar ...,3,Jagung Kuning segar,37.920520,[ayam],ayam,ayam,302.0
1905,Rendang Ayam Wini,1 ekor ayam ukuran kecil utuh (potong 8 bagian...,Cuci bersih ayam--Tumis bumbu rendang indofood...,7,Sagu Rendang,42.898937,[ayam],ayam,ayam,302.0
1095,Ayam ungkap bumbu serundeng,1 kg ayam--300 gr kelapa parut--daun salam--da...,"Ungkap ayam dengan bumbu halus, beri air tutup...",11,Ayam,14.215507,[ayam],ayam,ayam,302.0
448,Siomay Ayam Tempura,250 gr Ayam (blender)--150 gr Tempura (blender...,"Campurkan semua bahan, aduk sampai rata. Tes r...",8,Siomay,57.427437,"[ayam, siomay ]",ayam,ayam,302.0


In [41]:
df_recipe_cleaned = df_recipe[['title', 'ingredients','steps', 'loves', 'main_ingredient_clean', 'main_ingredient_calories']]

In [42]:
df_recipe_cleaned.columns

Index(['title', 'ingredients', 'steps', 'loves', 'main_ingredient_clean',
       'main_ingredient_calories'],
      dtype='object')

In [43]:
df_recipe_cleaned.sample(5)

,title,ingredients,steps,loves,main_ingredient_clean,main_ingredient_calories
1051,Tumis Tauge Ayam Wortel,4 siung bawah merah--2 butir bawang putih--1/2...,Goreng ayam lalu suwir - suwir--Rajang semua b...,5,ayam,302.0
276,Hainan Chicken - Ayam Rebus Hainan - Phak Cam Kee,"1 ekor ayam (baiknya ayam kampung, tp broiler ...",Ayam dibuang kepala & kakinya. Biarkan bulat u...,10,ayam,302.0
1431,Ayam bumbu rujak,1/2 kg ayam--10 buah cabe merah--8 buah rawit ...,Potong ayam sesuai selera dn cuci bersih--Gore...,7,ayam,302.0
1045,Nugget ayam keju,200 gram ayam giling--2 lembar roti tawar--2 b...,"Campurkan ayam giling,parutan keju,parutan wor...",2,keju,326.0
916,Ayam geprek 🍗🌶🌶,fillet Dada ayam--bawang putih haluskan--lada ...,Cuci bersih ayam kemudian lumuri ayam dengan b...,8,ayam,302.0


In [44]:
df_recipe_cleaned.shape

(1916, 6)

In [45]:
df_recipes = df_recipe_cleaned.copy()

In [46]:
df_recipes.iloc[0]['ingredients']

'1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk Nipis--2 Sdm Garam--3 Ruas Kunyit--7 Bawang Merah--7 Bawang Putih--10 Cabe Merah--10 Cabe Rawit Merah (sesuai selera)--3 Butir Kemiri--2 Batang Sereh--2 Lembar Daun Salam--2 Ikat Daun Kemangi--Penyedap Rasa--1 1/2 Gelas Air--'

In [47]:
ingredients_string = '1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk Nipis--2 Sdm Garam--3 Ruas Kunyit--7 Bawang Merah--7 Bawang Putih--10 Cabe Merah--10 Cabe Rawit Merah (sesuai selera)--3 Butir Kemiri--2 Batang Sereh--2 Lembar Daun Salam--2 Ikat Daun Kemangi--Penyedap Rasa--1 1/2 Gelas Air--'

ingredients_list = [item.strip() for item in ingredients_string.split('--') if item.strip()]

In [48]:
print(ingredients_list)

['1 Ekor Ayam Kampung (potong 12)', '2 Buah Jeruk Nipis', '2 Sdm Garam', '3 Ruas Kunyit', '7 Bawang Merah', '7 Bawang Putih', '10 Cabe Merah', '10 Cabe Rawit Merah (sesuai selera)', '3 Butir Kemiri', '2 Batang Sereh', '2 Lembar Daun Salam', '2 Ikat Daun Kemangi', 'Penyedap Rasa', '1 1/2 Gelas Air']


In [49]:
# Buat salinan dulu biar aman
df_recipe_split = df_recipes.copy()

In [50]:
# Ubah setiap string 'ingredients' jadi list berdasarkan pemisah '--'
df_recipe_split['ingredients_list'] = df_recipe_split['ingredients'].apply(lambda x: [i.strip() for i in str(x).split('--') if i.strip()])

In [51]:
# Temukan jumlah maksimum bahan untuk membuat kolom
max_ingredients = df_recipe_split['ingredients_list'].apply(len).max()

In [52]:
# Pisahkan jadi kolom-kolom baru
ingredients_expanded = df_recipe_split['ingredients_list'].apply(
    lambda x: pd.Series(x + [None]*(max_ingredients - len(x)))
)

In [53]:
# Tambahkan prefix nama kolom
ingredients_expanded.columns = [f'ingredient_{i+1}' for i in range(max_ingredients)]

In [54]:
# Gabungkan ke dataframe utama
df_recipe_split = pd.concat([df_recipe_split, ingredients_expanded], axis=1)

In [55]:
df_recipe_split.sample(5)

,title,ingredients,steps,loves,main_ingredient_clean,main_ingredient_calories,ingredients_list,ingredient_1,ingredient_2,ingredient_3,...,ingredient_32,ingredient_33,ingredient_34,ingredient_35,ingredient_36,ingredient_37,ingredient_38,ingredient_39,ingredient_40,ingredient_41
663,Kare ayam telor ala mamais,"1/2 kg ayam potong, potong sesuai selera, cuci...","Tumis bumbu halus bareng ama laos, sere, daun ...",3,ayam,302.0,"[1/2 kg ayam potong, potong sesuai selera, cuc...","1/2 kg ayam potong, potong sesuai selera, cuci...","5 bh telor rebus-kupas, goreng sebentar berkul...",2 ruas laos geprek,...,None,None,None,None,None,None,None,None,None,None
252,Opor Ayam,1 kg ayam--1 bks santan kara--Salam--Laos--Dau...,"Tumis bumbu halus bersama salam, laos, sereh d...",10,ayam,302.0,"[1 kg ayam, 1 bks santan kara, Salam, Laos, Da...",1 kg ayam,1 bks santan kara,Salam,...,None,None,None,None,None,None,None,None,None,None
1903,Ayam Crispy,1 ekor ayam (potong 8 bagian)--Bahan Marinase ...,Cuci bersih ayam. Lalu campur ayam dengan semu...,4,ayam,302.0,"[1 ekor ayam (potong 8 bagian), Bahan Marinase...",1 ekor ayam (potong 8 bagian),Bahan Marinase (Rendaman) :,Secukupnya garam,...,None,None,None,None,None,None,None,None,None,None
1668,Chicken Teriyaki Oriental Halal,1-2 bagian ayam fillet dada atau paha--1/2 bua...,Campur semua bahan saus Teriyaki dalam mangkuk...,4,chicken teriyaki oriental halal,NaN,"[1-2 bagian ayam fillet dada atau paha, 1/2 bu...",1-2 bagian ayam fillet dada atau paha,"1/2 buah bawang bombay, iris","2 sdm minyak, untuk memanggang",...,None,None,None,None,None,None,None,None,None,None
1077,Gulai ayam arya,Bahan halus;--5 ekor Ayam--5 bulat Bawang puti...,Bersihkan ayam potong sesuai selera. Setelah a...,9,ayam,302.0,"[Bahan halus;, 5 ekor Ayam, 5 bulat Bawang put...",Bahan halus;,5 ekor Ayam,5 bulat Bawang putih,...,None,None,None,None,None,None,None,None,None,None


In [56]:
# 1. Ambil semua kolom ingredients
ingredient_cols = [col for col in df_recipe_split.columns if col.startswith('ingredient_')]

In [57]:
# 2. Buat fungsi pencocokan kalori
def get_calories_from_nutrition(ingredient, nutrition_df):
    if pd.isna(ingredient):
        return None
    for nutri_name in nutrition_df['name']:
        if nutri_name.lower() in ingredient.lower():
            return nutrition_df[nutrition_df['name'] == nutri_name]['calories'].values[0]
    return None

In [58]:
# 3. Loop dan buat kolom kalori untuk setiap ingredient
for col in ingredient_cols:
    cal_col = col + '_calories'
    df_recipe_split[cal_col] = df_recipe_split[col].apply(lambda x: get_calories_from_nutrition(x, df_nutrition))

In [59]:
pd.set_option('display.max_columns', None)
df_recipe_split.sample(5)

,title,ingredients,steps,loves,main_ingredient_clean,main_ingredient_calories,ingredients_list,ingredient_1,ingredient_2,ingredient_3,ingredient_4,ingredient_5,ingredient_6,ingredient_7,ingredient_8,ingredient_9,ingredient_10,ingredient_11,ingredient_12,ingredient_13,ingredient_14,ingredient_15,ingredient_16,ingredient_17,ingredient_18,ingredient_19,ingredient_20,ingredient_21,ingredient_22,ingredient_23,ingredient_24,ingredient_25,ingredient_26,ingredient_27,ingredient_28,ingredient_29,ingredient_30,ingredient_31,ingredient_32,ingredient_33,ingredient_34,ingredient_35,ingredient_36,ingredient_37,ingredient_38,ingredient_39,ingredient_40,ingredient_41,ingredient_1_calories,ingredient_2_calories,ingredient_3_calories,ingredient_4_calories,ingredient_5_calories,ingredient_6_calories,ingredient_7_calories,ingredient_8_calories,ingredient_9_calories,ingredient_10_calories,ingredient_11_calories,ingredient_12_calories,ingredient_13_calories,ingredient_14_calories,ingredient_15_calories,ingredient_16_calories,ingredient_17_calories,ingredient_18_calories,ingredient_19_calories,ingredient_20_calories,ingredient_21_calories,ingredient_22_calories,ingredient_23_calories,ingredient_24_calories,ingredient_25_calories,ingredient_26_calories,ingredient_27_calories,ingredient_28_calories,ingredient_29_calories,ingredient_30_calories,ingredient_31_calories,ingredient_32_calories,ingredient_33_calories,ingredient_34_calories,ingredient_35_calories,ingredient_36_calories,ingredient_37_calories,ingredient_38_calories,ingredient_39_calories,ingredient_40_calories,ingredient_41_calories
1563,Tumis Ayam Bumbu Kuning,250 gr dada ayam fillet--1/4 bagian kol--2 bua...,"Panaskan minyak, tumis bumbu halus hingga mata...",4,ayam,302.0,"[250 gr dada ayam fillet, 1/4 bagian kol, 2 bu...",250 gr dada ayam fillet,1/4 bagian kol,2 buah wortel,3 batang daun bawang,Bumbu halus,4 siung bawang merah,4 siung bawang putih,2 cm kunyit,2 cm jahe,1 iris lengkuas,5 buah cabai (sesuai selera),Secukupnya merica bubuk,Secukupnya garam,Secukupnya gula,Secukupnya penyedap (opsional),Secukupnya air,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,302.0,NaN,NaN,29.0,NaN,39.0,95.0,63.0,51.0,NaN,NaN,359.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,NaN
1085,Ayam kecap,5 potong ayam--2 siung bawang merah--2 siung b...,"Goreng ayam hingga matang, angkat tiriskan--Ha...",7,ayam,302.0,"[5 potong ayam, 2 siung bawang merah, 2 siung ...",5 potong ayam,2 siung bawang merah,2 siung bawang putih,1/2 bh tomat,1 ruas lengkuas (dimemarkan),1 lembar daun salam,1 lembar daun jeruk,1 sdm saos,Secukupnya kecap garam merica,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,302.0,39.0,95.0,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,NaN
945,Ayam goreng spesial renyah rempah,"1/4 kg ayam--1 biji bumbu rempah (kunyit,jahe,...",Tumbuk halus semua bumbu.--Didihkan air dan ma...,6,ayam,302.0,"[1/4 kg ayam, 1 biji bumbu rempah (kunyit,jahe...",1/4 kg ayam,"1 biji bumbu rempah (kunyit,jahe,lengkuas)",2 siung bawang merah,2 siung bawang putih,Jeruk nipis (peras),1 sdm Garam,1 sdm Merica,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,302.0,51.0,39.0,95.0,37.0,NaN,359.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,NaN
785,Ayam Geprek,1/2 kg dada ayam--2 butir telur--5 sdm tepung ...,Ayam di cuci dan di rebus dengan air dan bumbu...,16,ayam,302.0,"[1/2 kg dada ayam, 2 butir telur, 5 sdm tepung...",1/2 kg dada ayam,2 butir telur,5 sdm tepu

In [60]:
df_recipe_split.to_csv('resep_dan_nutrisi.csv')

In [61]:
# Jumlahkan total kalori dari seluruh kolom kalori
calorie_cols = [col + '_calories' for col in ingredient_cols]
df_recipe_split['total_calories_estimated'] = df_recipe_split[calorie_cols].sum(axis=1, skipna=True)

In [62]:
df_recipe_split.sample(5)

,title,ingredients,steps,loves,main_ingredient_clean,main_ingredient_calories,ingredients_list,ingredient_1,ingredient_2,ingredient_3,ingredient_4,ingredient_5,ingredient_6,ingredient_7,ingredient_8,ingredient_9,ingredient_10,ingredient_11,ingredient_12,ingredient_13,ingredient_14,ingredient_15,ingredient_16,ingredient_17,ingredient_18,ingredient_19,ingredient_20,ingredient_21,ingredient_22,ingredient_23,ingredient_24,ingredient_25,ingredient_26,ingredient_27,ingredient_28,ingredient_29,ingredient_30,ingredient_31,ingredient_32,ingredient_33,ingredient_34,ingredient_35,ingredient_36,ingredient_37,ingredient_38,ingredient_39,ingredient_40,ingredient_41,ingredient_1_calories,ingredient_2_calories,ingredient_3_calories,ingredient_4_calories,ingredient_5_calories,ingredient_6_calories,ingredient_7_calories,ingredient_8_calories,ingredient_9_calories,ingredient_10_calories,ingredient_11_calories,ingredient_12_calories,ingredient_13_calories,ingredient_14_calories,ingredient_15_calories,ingredient_16_calories,ingredient_17_calories,ingredient_18_calories,ingredient_19_calories,ingredient_20_calories,ingredient_21_calories,ingredient_22_calories,ingredient_23_calories,ingredient_24_calories,ingredient_25_calories,ingredient_26_calories,ingredient_27_calories,ingredient_28_calories,ingredient_29_calories,ingredient_30_calories,ingredient_31_calories,ingredient_32_calories,ingredient_33_calories,ingredient_34_calories,ingredient_35_calories,ingredient_36_calories,ingredient_37_calories,ingredient_38_calories,ingredient_39_calories,ingredient_40_calories,ingredient_41_calories,total_calories_estimated
1038,Ayam Lada Hitam #NoMSG,1/2 Dada ayam--1 paprika merah yg besar--1 baw...,Potong dadu dada ayam + bawang bombai + paprik...,31,ayam,302.0,"[1/2 Dada ayam, 1 paprika merah yg besar, 1 ba...",1/2 Dada ayam,1 paprika merah yg besar,1 bawang bombai besar,8 bawang putih,3 cabai pedas,Secukupnya garam,Secukupnya lada hitam (bisa diganti merica),Secukupnya kecap,1 sendok maizena,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,302.0,NaN,NaN,95.0,NaN,NaN,359.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,NaN,802.0
1570,Ayam Kecap Kentang,1/4 kg daging ayam (blh lebih)--1/2 bh kentang...,Rebus daging ayam dan kentang dg daun salam sa...,7,ayam,302.0,"[1/4 kg daging ayam (blh lebih), 1/2 bh kentan...",1/4 kg daging ayam (blh lebih),1/2 bh kentang (blh lebih),1 lembar daun salam,1/2 bawang bombay,1 btg sereh,2 cm lengkuas,1 daun jeruk,2 cm jahe,Lada bubuk,"Gula, garam",Kecap manis,Penyedap rasa (sy pakai sejumput),Air utk merebus ayam,Daun bawang (optional),Bumbu halus :,4 siung bawang merah,2 siung bawang putih,1 bh kemiri,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,302.0,83.0,NaN,45.0,NaN,NaN,NaN,51.0,NaN,NaN,46.0,NaN,302.0,29.0,NaN,39.0,95.0,636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,NaN,1628.0
803,Tongseng Ayam,1 pasang dada ayam--Bumbu halus :--5 siung baw...,"Bahan bumbu halus diblender, bisa juga diuleg,...",6,ayam,302.0,"[1 pasang dada ayam, Bumbu halus :, 5 siung ba...",1 pasang dada ayam,Bumbu halus :,5 siung bawang merah,3 siung bawang putih,3 buah kemiri sangrai,1/2 sdt jinten,1/2 sdt lada putih bubuk,secukupnya Garam,Bumbu lainnya:,5 buah cengkeh,3 cm lengkuas digeprek,1 batang sereh geprek,5 lembar daun jeruk,3 buah kapulaga,3 sdm kecap manis,Pelengkap:,Tomat potong-potong,Kol potong kasar,Bawang bombay (tambahan dari saya),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,302.0,NaN,39.0,95.0,636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,NaN,1163.0
1

In [63]:
# Ambil semua kolom ingredients
ingredient_cols = [col for col in df_recipe_split.columns if col.startswith('ingredient_')]

# Ganti NaN menjadi 0 pada kolom-kolom tersebut
df_recipe_split[ingredient_cols] = df_recipe_split[ingredient_cols].fillna(0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9676\244935570.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_recipe_split[ingredient_cols] = df_recipe_split[ingredient_cols].fillna(0)


In [64]:
# Kolom kalori dari setiap ingredient
ingredient_calorie_cols = [col for col in df_recipe_split.columns if col.startswith('ingredient_') and '_calories' in col]

# Ganti NaN menjadi 0 juga untuk kolom kalorinya
df_recipe_split[ingredient_calorie_cols] = df_recipe_split[ingredient_calorie_cols].fillna(0)

In [65]:
df_recipe_split.sample(5)

,title,ingredients,steps,loves,main_ingredient_clean,main_ingredient_calories,ingredients_list,ingredient_1,ingredient_2,ingredient_3,ingredient_4,ingredient_5,ingredient_6,ingredient_7,ingredient_8,ingredient_9,ingredient_10,ingredient_11,ingredient_12,ingredient_13,ingredient_14,ingredient_15,ingredient_16,ingredient_17,ingredient_18,ingredient_19,ingredient_20,ingredient_21,ingredient_22,ingredient_23,ingredient_24,ingredient_25,ingredient_26,ingredient_27,ingredient_28,ingredient_29,ingredient_30,ingredient_31,ingredient_32,ingredient_33,ingredient_34,ingredient_35,ingredient_36,ingredient_37,ingredient_38,ingredient_39,ingredient_40,ingredient_41,ingredient_1_calories,ingredient_2_calories,ingredient_3_calories,ingredient_4_calories,ingredient_5_calories,ingredient_6_calories,ingredient_7_calories,ingredient_8_calories,ingredient_9_calories,ingredient_10_calories,ingredient_11_calories,ingredient_12_calories,ingredient_13_calories,ingredient_14_calories,ingredient_15_calories,ingredient_16_calories,ingredient_17_calories,ingredient_18_calories,ingredient_19_calories,ingredient_20_calories,ingredient_21_calories,ingredient_22_calories,ingredient_23_calories,ingredient_24_calories,ingredient_25_calories,ingredient_26_calories,ingredient_27_calories,ingredient_28_calories,ingredient_29_calories,ingredient_30_calories,ingredient_31_calories,ingredient_32_calories,ingredient_33_calories,ingredient_34_calories,ingredient_35_calories,ingredient_36_calories,ingredient_37_calories,ingredient_38_calories,ingredient_39_calories,ingredient_40_calories,ingredient_41_calories,total_calories_estimated
503,Ayam bakar bumbu bbq,1 / 750 gram ekor ayam--4 sdm kecap--5 sdm bub...,Setelah ayam dibersihkan.lumuri dgn air jeruk ...,14,ayam,302.0,"[1 / 750 gram ekor ayam, 4 sdm kecap, 5 sdm bu...",1 / 750 gram ekor ayam,4 sdm kecap,5 sdm bubuk baberque,3 sdm kecap asin,3 sdm saos tomat,4 bawang putih,2 sdm saos teriyaki,1 saset bubuk merica,2 sdm margarine,1/4 gelas air,3 sdm air asam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,302.0,46.0,0.0,46.0,98.0,95.0,0.0,359.0,720.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,1666.0
1244,Soto bening ayam,1/2 kg ayam dada--bumbu halus :--4 siung bawan...,"Cuci ayam smp bersih,rebus ayam dgn air mendid...",2,ayam,302.0,"[1/2 kg ayam dada, bumbu halus :, 4 siung bawa...",1/2 kg ayam dada,bumbu halus :,4 siung bawang putih,3 siung bawang merah,1 sdt merica,1 sdt ketumbar,1 ruas kunyit,1 ruas jahe,3 biji kemiri,bumbu pelengkap:,"1 ruas lengkuas,geprek",2 lembar daun jeruk,"1 batang serai,geprek",2 lembar daun salam,"1 batang daun bawang ikat,1buah tomat potong",bahan pelengkap:,1/2 kg kentang,tauge,seledri,bawang goreng,acar timun,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,302.0,0.0,95.0,39.0,359.0,404.0,63.0,51.0,636.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,83.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,2081.0
1741,Ayam pekcamke (chicken hainanese),1 ekor ayam kampung--Jeruk nipis dan garam utk...,Lumuri ayam dengan jeruk nipis dan garam...bia...,9,ayam,302.0,"[1 ekor ayam kampung, Jeruk nipis dan garam ut...",1 ekor ayam kampung,Jeruk nipis dan garam utk melumuri ayam,1 ruas jahe geprek,1 btg daun bawang kecil diikat,Air es utk merendam,Air utk memasak,Minyak wijen,Kecap asin,Bawang putih,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,302.0,302.0,51.0,29.0,0.0,0.0,902.0,46.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,1727.0
1724,Ayam Rica Kecap,1 kg ayam dipotong sesuai selera--Bumbu Halus ...,Panaskan minyak secukupnya kemudian Tumis bumb...,6,ayam,302.0,"[1 kg ayam dipotong sesuai selera, Bumbu Halus...",1 kg ayam dipotong sesuai selera,Bumbu Halus :,3 Biji Lombok Merah,10 siung bawang merah,10 siung bawang putih,15 biji Lombok biasa (kalau suka pedas banget...,Jahe,Laos di memar

In [66]:
df_recipe_split.to_csv('resep_dan_nutrisi.csv')

In [67]:
df_recipe_split.columns

Index(['title', 'ingredients', 'steps', 'loves', 'main_ingredient_clean',
       'main_ingredient_calories', 'ingredients_list', 'ingredient_1',
       'ingredient_2', 'ingredient_3', 'ingredient_4', 'ingredient_5',
       'ingredient_6', 'ingredient_7', 'ingredient_8', 'ingredient_9',
       'ingredient_10', 'ingredient_11', 'ingredient_12', 'ingredient_13',
       'ingredient_14', 'ingredient_15', 'ingredient_16', 'ingredient_17',
       'ingredient_18', 'ingredient_19', 'ingredient_20', 'ingredient_21',
       'ingredient_22', 'ingredient_23', 'ingredient_24', 'ingredient_25',
       'ingredient_26', 'ingredient_27', 'ingredient_28', 'ingredient_29',
       'ingredient_30', 'ingredient_31', 'ingredient_32', 'ingredient_33',
       'ingredient_34', 'ingredient_35', 'ingredient_36', 'ingredient_37',
       'ingredient_38', 'ingredient_39', 'ingredient_40', 'ingredient_41',
       'ingredient_1_calories', 'ingredient_2_calories',
       'ingredient_3_calories', 'ingredient_4_calories',

In [68]:
df_recipe_revised = df_recipe_split[['title','steps', 'loves', 'ingredients_list', 'total_calories_estimated']]

In [69]:
df_recipe_revised.head(5)

,title,steps,loves,ingredients_list,total_calories_estimated
0,Ayam Woku Manado,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,"[1 Ekor Ayam Kampung (potong 12), 2 Buah Jeruk...",1218.0
1,Ayam goreng tulang lunak,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,[1 kg ayam (dipotong sesuai selera jangan keci...,1802.0
2,Ayam cabai kawin,Panaskan minyak di dalam wajan. Setelah minyak...,2,"[1/4 kg ayam, 3 buah cabai hijau besar, 7 buah...",436.0
3,Ayam Geprek,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,"[250 gr daging ayam (saya pakai fillet), Secuk...",652.0
4,Minyak Ayam,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,"[400 gr kulit ayam & lemaknya, 8 siung bawang ...",852.0


In [70]:
consumer_profile

,gender,age_group,prefer_traditional,prefer_healthy
0,Female,26–35,Yes,No
1,Female,36–45,No,No
2,Male,18–25,No,No
3,Female,56+,Yes,No
4,Male,36–45,Yes,No
...,...,...,...,...
705,Female,56+,No,No
706,Male,46–55,No,No
707,Male,26–35,No,No
708,Male,26–35,No,No


In [71]:
consumer_profile = consumer_profile.reset_index(drop=True)
consumer_profile['user_id'] = consumer_profile.index

In [72]:
consumer_profile['user_id'] = consumer_profile.index + 1

In [73]:
user = consumer_profile.iloc[0]

In [74]:
user

gender                Female
age_group              26–35
prefer_traditional       Yes
prefer_healthy            No
user_id                    1
Name: 0, dtype: object

In [75]:
df_recipe_revised.to_csv('Revisi Resep Kostum Nutrisi.csv')
consumer_profile.to_csv("Profil Konsumen.csv")

Definisikan kriteria filtering resep:


Jika prefer_healthy == Yes → Pilih resep dengan kalori rendah (misal di bawah median).

Jika prefer_traditional == Yes → Pilih resep dengan nama mengandung kata seperti: 'Ayam', 'Soto', 'Rendang', 'Woku', 'Gulai', 'Nasi', dsb.

Jika prefer_healthy == No → Tidak perlu filter kalori.

Jika prefer_traditional == No → Boleh pilih semua resep.

In [76]:
def recommend_recipe(user_profile, df_recipe_revised):
    traditional_keywords = ['ayam', 'soto', 'rendang', 'woku', 'gulai', 'nasi', 'opor', 'rawon']
    calorie_threshold = df_recipe_revised['total_calories_estimated'].median()
    
    # Filter awal semua resep
    filtered = df_recipe_revised.copy()
    
    # Filter berdasarkan preferensi sehat
    if user_profile['prefer_healthy'] == 'Yes':
        filtered = filtered[filtered['total_calories_estimated'] <= calorie_threshold]
    
    # Filter berdasarkan preferensi tradisional
    if user_profile['prefer_traditional'] == 'Yes':
        filtered = filtered[filtered['title'].str.lower().str.contains('|'.join(traditional_keywords))]

    # Urutkan berdasarkan likes terbanyak
    recommended = filtered.sort_values(by='loves', ascending=False).head(5)
    
    return recommended[['title', 'loves', 'total_calories_estimated']]

#### Uji sistem rekomendasi pada konsumer 0

In [77]:
user = consumer_profile.iloc[0]
recommendation = recommend_recipe(user, df_recipe_revised)

In [78]:
recommendation

,title,loves,total_calories_estimated
1635,Ayam Goreng Bawang Khas Batam,237,1408.0
1065,Ayam Goreng Kecap,218,881.0
1690,Mie Ayam,194,1960.0
1471,"Menu dinner trio tumis - pokcoy, ayam bawang p...",170,1369.0
1466,Ayam Woku Manado,169,1295.0


In [79]:
all_recommendations = []

for idx, user in consumer_profile.iterrows():
    recs = recommend_recipe(user, df_recipe_revised)  # pastikan fungsi ini menerima user (row) dan df resep
    for _, row in recs.iterrows():
        all_recommendations.append({
            'user_id': user['user_id'],  # gunakan user_id dari kolom, bukan index
            'recipe_title': row['title'],
            'loves': row['loves'],
            'total_calories_estimated': row['total_calories_estimated']
        })

df_recommendation_result = pd.DataFrame(all_recommendations)

In [80]:
df_recommendation_result

,user_id,recipe_title,loves,total_calories_estimated
0,1,Ayam Goreng Bawang Khas Batam,237,1408.0
1,1,Ayam Goreng Kecap,218,881.0
2,1,Mie Ayam,194,1960.0
3,1,"Menu dinner trio tumis - pokcoy, ayam bawang p...",170,1369.0
4,1,Ayam Woku Manado,169,1295.0
...,...,...,...,...
3545,710,Ayam Goreng Bawang Khas Batam,237,1408.0
3546,710,Ayam Goreng Kecap,218,881.0
3547,710,Mie Ayam,194,1960.0
3548,710,"Menu dinner trio tumis - pokcoy, ayam bawang p...",170,1369.0


# Streamlit Model

In [81]:
pip install streamlit pandas

Note: you may need to restart the kernel to use updated packages.


In [82]:
# recommend.py
def recommend_recipe(user, df_recipe_revised):
    # Contoh aturan sederhana:
    traditional = user['prefer_traditional'] == 'Yes'
    healthy = user['prefer_healthy'] == 'Yes'

    df_filtered = df_recipe_revised.copy()

    if healthy:
        df_filtered = df_filtered[df_filtered['total_calories_estimated'] < 500]

    if traditional:
        df_filtered = df_filtered[df_filtered['title'].str.contains('Ayam|Nasi|Soto|Goreng', case=False)]

    # Urutkan berdasarkan popularitas
    df_filtered = df_filtered.sort_values(by='loves', ascending=False)

    return df_filtered.head(5)  # top 5 rekomendasi
